In [1]:
from gradients import *

In [2]:
SUBJECTS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
17, 18, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 33,
35, 36, 38, 39, 40, 42, 43, 44, 45, 46, ]
# excluded subjects: 41, 19, 32, 27, 34, 37,
#todo why?

In [9]:
# EPICS = ['rs', 'bl', 'lrn', 'lrne', 'lrnl']

EPICS = ['baseline', 'early_learning', 'late_learning']
# rest and learning are removed in make_gradients

In [3]:
NUM_COMPONENTS = 3

---
reference

In [ ]:
REF_EPIC = 'baseline'

In [5]:
data_rs = load_data(subj=1, epic='rest')
# just took a subj to make atlas
surf_labels, mask_removed = load_atlas(data_rs)
# make average corr_mat

In [ ]:
global_corr_mat = np.zeros((998, 998))
for s in SUBJECTS:
    data_baseline = load_data(subj=s, epic=REF_EPIC)
    global_corr_mat += make_mat(data_baseline)
global_corr_mat /= len(SUBJECTS)

In [7]:
# make the global gradient model
DIM_RED_APPROACH = 'pca'
global_gradient_reference = GradientMaps(random_state=0, approach=DIM_RED_APPROACH)
global_gradient_reference.fit(global_corr_mat, sparsity=0.9)
#todo ? sparsity

GradientMaps(approach='pca', random_state=0)

In [10]:
grads = [{k:{} for k in EPICS} for _ in range(NUM_COMPONENTS)]

for subject in SUBJECTS:
    gm = make_gradients(subj=subject, gm_ref=global_gradient_reference)
    for component in range(NUM_COMPONENTS):
        for epic in EPICS:
            grad = gm.aligned_[EPICS.index(epic)][:, component]
            grads[component][epic][subject] = grad

---
the overall shitty structure of data:

In [11]:
len(grads)

3

In [12]:
grads[0].keys()

dict_keys(['baseline', 'early_learning', 'late_learning'])

In [13]:
grads[0]['baseline'].keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 33, 35, 36, 38, 39, 40, 42, 43, 44, 45, 46])

---
make the super dataframe

In [18]:
v = grads[0]['baseline'][1]
v = pd.DataFrame(v, columns=['value'])
v.index = v.index.set_names(['region_idx'])
v = v.reset_index()
v

,region_idx,value
0,0,-0.774317
1,1,-1.407623
2,2,-1.905399
3,3,-1.225759
4,4,-0.971639
...,...,...
993,993,-1.778768
994,994,-1.534647
995,995,-1.945736
996,996,0.558007


In [20]:
df = pd.DataFrame()
for comp in range(NUM_COMPONENTS):
    for epic in EPICS:
        for subj in SUBJECTS:
            values = grads[comp][epic][subj]
            d = pd.DataFrame(values, columns=['value'])
            d.index = d.index.set_names(['region_idx'])
            d = d.reset_index()
            d['subj'] = subj
            d['epic'] = epic
            d['comp'] = comp
            df = pd.concat([df, d], axis=0)

In [22]:
df

,region_idx,value,subj,epic,comp
0,0,-0.774317,1,baseline,0
1,1,-1.407623,1,baseline,0
2,2,-1.905399,1,baseline,0
3,3,-1.225759,1,baseline,0
4,4,-0.971639,1,baseline,0
...,...,...,...,...,...
993,993,0.586871,46,late_learning,2
994,994,1.364654,46,late_learning,2
995,995,-0.143286,46,late_learning,2
996,996,0.249002,46,late_learning,2


In [24]:
df.shape[0] /998/40/3/3

1.0

In [ ]:
df.groupby(['subj', 'epic', 'region_idx']).agg({'value': np.mean})

value
subj epic          region_idx          
1    baseline      0           0.710846
                   1           0.126286
                   2          -0.057745
                   3           0.448429
                   4           0.473483
...                                 ...
46   late_learning 993        -0.835918
                   994        -1.189531
                   995        -0.858283
                   996         0.224819
                   997        -0.038616

[119760 rows x 1 columns]

In [ ]:
# fliter, transform, aggregate

In [25]:
ecc = df[df.comp.isin([0, 1, 2])]\
    .assign(v2=lambda r: r['value']**2, aixs=1)\
        .groupby(['subj', 'epic', 'region_idx']).agg({'v2': sum})\
            .v2.apply(np.sqrt).reset_index()\
                .rename(columns={'v2': 'value'})
ecc['comp'] = 'ecc'

ecc

,subj,epic,region_idx,value,comp
0,1,baseline,0,3.738824,ecc
1,1,baseline,1,1.606313,ecc
2,1,baseline,2,2.232096,ecc
3,1,baseline,3,2.600396,ecc
4,1,baseline,4,2.093912,ecc
...,...,...,...,...,...
119755,46,late_learning,993,1.497172,ecc
119756,46,late_learning,994,2.645587,ecc
119757,46,late_learning,995,0.922904,ecc
119758,46,late_learning,996,2.302812,ecc


In [29]:
df.shape[0] // NUM_COMPONENTS, ecc.shape[0]

(119760, 119760)

In [30]:
df = pd.concat([df, ecc], axis=0)

In [31]:
# df.to_csv('subjects_gradients_ecc.csv')

In [33]:
df.comp.unique()

array([0, 1, 2, 'ecc'], dtype=object)